# PS1: Your first library-free neural network!  

Advanced Learning 2024


For SUBMISSION:   

Please upload the complete and executed `ipynb` to your git repository. Verify that all of your output can be viewed directly from github, and provide a link to that git file below.

~~~
STUDENT ID: 342766763
~~~

~~~
STUDENT GIT LINK: https://github.com/mickaelAssous/52025
~~~
In Addition, don't forget to add your ID to the files:    
  
`PS1_Part2_HelloNN_2024_ID_[000000000].html`   


In [1]:
import numpy as np # You are allowed to use  only numpy.
import time


**Welcome**.   

In this part of the problem set you are set to build a complete and flexible neural network.  
This neural network will be library free (in the sense that we won't use PyTorch/Tensorflow/etc.).   

Let's do a quick review of the basic neural-network components:  


*   *Layer* - can be fully connected (dense/hidden), convolution, etc.
  * Forward propagation- the layer outputs the next layer's input
  * Backward propagation- the layer also outputs the gradient descent update
*   *Activation* Layer (e.g. ReLU) - there are no parameters, only gradients with respect to the input. We want to compute both the gradient w.r.t the parameters of the layer and to create the gradient with respect to the layer's inputs
   * *Forward propagation*- the layer outputs the next layer's input
   * *Backward propagation*- the layer also outputs the gradient descent update
*   *Loss Function* : how our model  quantifies the difference between the predicted outputs the actual (target) values  
*   *Network Wrapper*-  wraps our components together as a trainable model.






Useful resource:  
* Gradient descent for neural networks [cheat sheet](https://moodle4.cs.huji.ac.il/hu23/mod/resource/view.php?id=402297).
* Neural network architecture [cheat sheet](https://moodle4.cs.huji.ac.il/hu23/mod/url/view.php?id=402298).

### 0. Loading data

You are going to test and evaluate your home-made network on the `mnist` dataset.   
The MNIST dataset is a large dataset of handwritten digits that is commonly used for training various image and vision models.

In [2]:
from keras.datasets import mnist
from keras.utils import to_categorical
# load MNIST from server
# Using a standard library (keras.datasets) to load the mnist data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


#### Data transformations





In [3]:
# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# One-hot encoding of the output.
# Currently a number in range [0,9]; Change into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = to_categorical(y_train)
# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)

### 1. Network's Components

Please fill-in the missing code in the code boxes below (only where  `#### SOLUTION REQUIRED ####` is specified).   

In [4]:

# This class is a general layer primitive, defining that each instance must
# have an (input,output) parameters, and 2 functions: forward+backward propogation
class Layer_Primitive:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

#### Fully Connected Layer

A fully-connected layer (a.k.a. affine, dense,linear layer) connects every input neuron to every output neuron.   
It has 2 parameters: (input, output).   
You need to define (code) the following:
* its initialization weights with random weights.
* the forward propogation calculation (as shown in class).
* the backward propogation gradients calculation (given output, as shown in class).

Parameters must be intitialized with some values. There are many ways to initialize the weights, and you are encouraged to do a quick research about the common methods. Any commonly used method will be accepted.  

1.1 (20 pts)

In [5]:
#### SOLUTION REQUIRED ####


# inherit from base class Layer
class Affine_Layer(Layer_Primitive):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # Random initialization of weights and biases
        # Using Xavier Initialization for weights
        limit = np.sqrt(6 / (input_size + output_size))  # Xavier initialization range
        self.weights = np.random.uniform(-limit, limit, (input_size, output_size))
        self.bias = np.zeros((1, output_size))  # Biases initialized to zero

    # Forward propagation: Y = XW + B
    def forward_propagation(self, input_data):
        self.input = input_data  # Store input for backpropagation
        self.output = np.dot(self.input, self.weights) + self.bias  # Linear transformation
        return self.output

    # Backward propagation: computes dE/dW, dE/dB and dE/dX
    def backward_propagation(self, output_grad, learning_rate):
        # dE/dW = X^T * dE/dY
        weights_error = np.dot(self.input.T, output_grad)
        # dE/dB = Sum of dE/dY across samples
        bias_error = np.sum(output_grad, axis=0, keepdims=True)
        # dE/dX = dE/dY * W^T
        input_error = np.dot(output_grad, self.weights.T)

        # Update weights and biases
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * bias_error

        return input_error



#### Activation layers

Activation functions are often a non-linear functions that aid in how well the network model adapts to and learns  the training dataset. The choice of activation function in the output layer will define the type of predictions the model can make.  



In [6]:
# inherit from base class Layer
class ActivationLayer(Layer_Primitive):
    def __init__(self, activation, activation_grad):
        self.activation = activation
        self.activation_grad = activation_grad

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_grad=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_grad, learning_rate):
        return self.activation_grad(self.input) * output_grad



You need to define (code) the following via different functions:
* the forward propogation calculation (as shown in class).
* the backward propogation gradients calculation (given output, as shown in class).

1.2 (20 pts)

In [7]:
#### SOLUTION REQUIRED ####

# activation functions and their derivatives:

def tanh(x):
    # Hyperbolic tangent: outputs values in the range (-1, 1)
    return np.tanh(x)

def tanh_grad(x):
    # Derivative of tanh: 1 - tanh(x)^2
    return 1 - np.tanh(x)**2

def relu(x):
    # ReLU: max(0, x)
    return np.maximum(0, x)

def relu_grad(x):
    # Derivative of ReLU: 1 if x > 0, else 0
    return (x > 0).astype(float)

def sigmoid(x):
    # Sigmoid: outputs values in the range (0, 1)
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    # Derivative of sigmoid: sigmoid(x) * (1 - sigmoid(x))
    s = sigmoid(x)
    return s * (1 - s)


#### Loss function

1.3 (10 pts)

In [8]:
#### SOLUTION REQUIRED ####

# Loss function (Mean Squared Error) and its derivative

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mse_grad(y_true, y_pred):
    return -2 * (y_true - y_pred) / y_true.size



#### Putting everything together

1.4 (10 pts)

In [9]:
#### SOLUTION REQUIRED (in `predict`) ####

class MyNetwork:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_grad = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use_loss(self, loss, loss_grad):
        self.loss = loss
        self.loss_grad = loss_grad


    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                grad = self.loss_grad(y_train[j], output)
                for layer in reversed(self.layers):
                    grad = layer.backward_propagation(grad, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('Training epoch %d/%d   error=%f' % (i+1, epochs, err))

    # predict output for given input
    def predict(self, x_test,y_test=np.array([])):
        if y_test.size:
           assert len(x_test)==len(y_test) # if Y is given
        # sample dimension first
        samples = len(x_test)
        result = []
        loss = 0
        correct = 0
        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = x_test[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
            # ONLY IF LABELS ARE GIVEN (Y):
            if y_test.size:
                # Evaluate the output against Y,
                # calculate loss against Y, add to `loss`:
                loss += self.loss(y_test[i], output)
                target = y_test[i]
                # Evaluate the label of the output against real, and if identical,
                # add +1 to `correct`:
                if np.argmax(output) == np.argmax(target):
                   correct += 1
        if y_test.size:
            mean_loss = loss/samples

            print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.
                  format(mean_loss, correct, samples,100. * correct / samples))

        return result




## 2. Testing Your Neural Network

### Defining our main neural network architecture

Define your network's architecture:  
(Please rationalize your choice of activation funciton.)
* first affine layer that takes your input and outputs 128 nodes
* `tanh/relu/sigmoid` activation layer following the first affine layer
* second affine layer that takes the first layer's input and outputs 64 nodes
* `tanh/relu/sigmoid` activation layer following the second affine layer
* third affine layer that takes your second layer's input and outputs nodes in the size of the Y labels.
* `tanh/relu/sigmoid` activation layer following the last affine layer


2.1 (5 pts)

In [10]:
#### SOLUTION REQUIRED (in `predict`) ####

# Define Network Architecture
net = MyNetwork()
# First Affine Layer (Input -> 128)
net.add(Affine_Layer(784, 128))
net.add(ActivationLayer(relu, relu_grad))

# Second Affine Layer (128 -> 64)
net.add(Affine_Layer(128, 64))
net.add(ActivationLayer(relu, relu_grad))

# Third Affine Layer (64 -> 10, Output)
net.add(Affine_Layer(64, 10))
net.add(ActivationLayer(sigmoid, sigmoid_grad))


### Training!

In [11]:

# While developing, it is recommended to train your model on a subset of the data... / or low epochs.
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use_loss(mse, mse_grad)
epoch_num = 20
lr = 0.01
t1 = time.time()
net.fit(x_train, y_train, epochs=epoch_num, learning_rate=lr)
print(f"Total process time: {round(time.time() - t1,3)}")


Training epoch 1/20   error=0.033791
Training epoch 2/20   error=0.014619
Training epoch 3/20   error=0.011781
Training epoch 4/20   error=0.010129
Training epoch 5/20   error=0.008972
Training epoch 6/20   error=0.008093
Training epoch 7/20   error=0.007392
Training epoch 8/20   error=0.006808
Training epoch 9/20   error=0.006313
Training epoch 10/20   error=0.005886
Training epoch 11/20   error=0.005513
Training epoch 12/20   error=0.005184
Training epoch 13/20   error=0.004890
Training epoch 14/20   error=0.004625
Training epoch 15/20   error=0.004384
Training epoch 16/20   error=0.004163
Training epoch 17/20   error=0.003963
Training epoch 18/20   error=0.003778
Training epoch 19/20   error=0.003608
Training epoch 20/20   error=0.003452
Total process time: 1054.771


### Evaluation

Exciting! Now is the time to test your model.   

    May the gradients be always in your favor.

In [12]:
output = net.predict(x_test ,y_test )


Test set: Avg. loss: 0.0047, Accuracy: 9737/10000 (97%)



## 3. Benchmarking against PyTorch

How well your model performs against a similar-architecture PyTorch model?   
It is time to find out:

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset

#### Prepare the data as tensors using PyTorch DataLoader:

In [14]:
t_train =  TensorDataset(torch.Tensor(x_train),torch.Tensor(y_train))
t_test =  TensorDataset(torch.Tensor(x_test),torch.Tensor(y_test))
train_loader = torch.utils.data.DataLoader(dataset=t_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=t_test, batch_size=64, shuffle=False)

Define a `PyTorchNet` class with an identical architecture you used in your home-made network.

3.1 (10 pts)

In [15]:
#### SOLUTION REQUIRED  ####

class PyTorchNet(nn.Module):
    def __init__(self):
        super(PyTorchNet, self).__init__()
        input_size = 28 * 28
        num_classes = 10

        self.fc1 = nn.Linear(input_size, 128)
        self.activ1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.activ1 = nn.ReLU()
        self.fc3 = nn.Linear(64, num_classes)
        self.activ1 = nn.Sigmoid()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.activ1(self.fc1(x))
        x = self.activ1(self.fc2(x))
        x = self.activ1(self.fc3(x))
        return x

In [17]:

# Train the model
num_epochs = 20
pt_learning_rate = 0.01
pt_network = PyTorchNet()
optimizer = torch.optim.Adam(pt_network.parameters(), lr=pt_learning_rate)
criterion = nn.MSELoss()

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = pt_network(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # A handy printout:
        if (i + 1) % 500 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/20], Step [500/938], Loss: 0.0499
Epoch [2/20], Step [500/938], Loss: 0.0114
Epoch [3/20], Step [500/938], Loss: 0.0044
Epoch [4/20], Step [500/938], Loss: 0.0075
Epoch [5/20], Step [500/938], Loss: 0.0050
Epoch [6/20], Step [500/938], Loss: 0.0096
Epoch [7/20], Step [500/938], Loss: 0.0028
Epoch [8/20], Step [500/938], Loss: 0.0030
Epoch [9/20], Step [500/938], Loss: 0.0084
Epoch [10/20], Step [500/938], Loss: 0.0018
Epoch [11/20], Step [500/938], Loss: 0.0030
Epoch [12/20], Step [500/938], Loss: 0.0023
Epoch [13/20], Step [500/938], Loss: 0.0037
Epoch [14/20], Step [500/938], Loss: 0.0072
Epoch [15/20], Step [500/938], Loss: 0.0032
Epoch [16/20], Step [500/938], Loss: 0.0042
Epoch [17/20], Step [500/938], Loss: 0.0035
Epoch [18/20], Step [500/938], Loss: 0.0001
Epoch [19/20], Step [500/938], Loss: 0.0051
Epoch [20/20], Step [500/938], Loss: 0.0046


Evaluation:

In [18]:
pt_network.eval()
test_losses = []
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = pt_network(data)
        test_loss += criterion(output, target,)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.max(1,keepdim=True)[1]).sum()

test_loss /= len(test_loader.dataset)
test_losses.append(test_loss)
print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  test_loss, correct, len(test_loader.dataset),
  100. * correct / len(test_loader.dataset)))


Test set: Avg. loss: 0.0001, Accuracy: 9686/10000 (97%)



3.2 (10 pts)

Time for some questions:
1. Which one of the models performed better? Why?
2. Which one of the models performed faster? Why?  
3. What would you change in your network's architecture?   
4. What would you change in your model's solution algorithm?

Write your solutions here:
1. Both models achieved similar performances, with around 97% accuracy. However, the PyTorch model had a slightly lower average loss (0.0001 vs. 0.0047 for the custom network). PyTorch uses advanced built-in optimizations like Adam (an enhanced gradient descent algorithm) and more precise calculations for backpropagation.

2. The PyTorch model was faster for several reasons: PyTorch is optimized for matrix calculations with specialized libraries designed to accelerate neural networks. Additionally, my custom network calculates backpropagation manually layer by layer, which is slower than PyTorch's vectorized approach.

3. To improve the architecture of my network, we could add a new hidden layer. By increasing the depth (more layers and neurons), the model can learn more complex relationships in the data.

4. We can change the loss function in our algorithm.

## 4. The Network Wars!

Here is your chance to play with your model's architecture in order to break your own benchmark set eariler.  
You can add/remove layers, play with their sizes, types, etc.   
You can add a new loss if you wish, or anything else that will fairly give your model an advantage over base.  

4.1 (15 pts)

In [19]:
#### SOLUTION REQUIRED  ####

class MyImprovedNetwork:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_grad = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use_loss(self, loss, loss_grad):
        self.loss = loss
        self.loss_grad = loss_grad

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for epoch in range(epochs):
            err = 0
            for i in range(samples):
                # forward propagation
                output = x_train[i]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                 # compute loss (for display purpose only)
                err += self.loss(y_train[i], output)

                # backward propagation
                grad = self.loss_grad(y_train[i], output)
                for layer in reversed(self.layers):
                    grad = layer.backward_propagation(grad, learning_rate)

            err /= samples
            print(f"Epoch {epoch + 1}/{epochs}, Error={err:.4f}")

    # predict output for given input
    def predict(self, x_test, y_test=None):
        results = []
        loss = 0
        correct = 0
        for i in range(len(x_test)):
            output = x_test[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            results.append(output)

            if y_test is not None:
                loss += self.loss(y_test[i], output)
                if np.argmax(output) == np.argmax(y_test[i]):
                    correct += 1

        if y_test is not None:
            print(f"Test Loss={loss / len(x_test):.4f}, Accuracy={correct}/{len(x_test)} "
                  f"({100 * correct / len(x_test):.2f}%)")
        return results


In [20]:
# Define Network Architecture
net = MyImprovedNetwork()

# First Affine Layer (Input -> 256)
net.add(Affine_Layer(784, 256))
net.add(ActivationLayer(relu, relu_grad))

# Second Affine Layer (256 -> 128)
net.add(Affine_Layer(256, 128))
net.add(ActivationLayer(relu, relu_grad))

# Third Affine Layer (128 -> 64)
net.add(Affine_Layer(128, 64))
net.add(ActivationLayer(relu, relu_grad))

# Fourth Affine Layer (64 -> 10, output)
net.add(Affine_Layer(64, 10))
net.add(ActivationLayer(sigmoid, sigmoid_grad))


In [22]:
net.use_loss(mse, mse_grad)

# train
epochs = 20
learning_rate = 0.01
t1 = time.time()
net.fit(x_train, y_train, epochs=epochs, learning_rate=learning_rate)
print(f"Total process time: {round(time.time() - t1,3)}")


Epoch 1/20, Error=0.0164
Epoch 2/20, Error=0.0112
Epoch 3/20, Error=0.0090
Epoch 4/20, Error=0.0075
Epoch 5/20, Error=0.0065
Epoch 6/20, Error=0.0057
Epoch 7/20, Error=0.0050
Epoch 8/20, Error=0.0045
Epoch 9/20, Error=0.0041
Epoch 10/20, Error=0.0037
Epoch 11/20, Error=0.0034
Epoch 12/20, Error=0.0031
Epoch 13/20, Error=0.0029
Epoch 14/20, Error=0.0026
Epoch 15/20, Error=0.0024
Epoch 16/20, Error=0.0023
Epoch 17/20, Error=0.0021
Epoch 18/20, Error=0.0020
Epoch 19/20, Error=0.0018
Epoch 20/20, Error=0.0017
Total process time: 2314.946


In [23]:
# Test the model
output = net.predict(x_test, y_test)

Test Loss=0.0037, Accuracy=9795/10000 (97.95%)


In [24]:
!jupyter nbconvert --to html /PS1_Part2_HelloNN_2024_342766763.ipynb

[NbConvertApp] WARNING | pattern '/PS1_Part2_HelloNN_2024_342766763.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_y